# GPU Check

In [ ]:
import GPUtil
GPUs = GPUtil.getGPUs()
for gpu in GPUs:
  print(gpu.name, gpu.memoryTotal)

# Imports

In [ ]:
from search_eval.eval_generic import SGLDES
from search_eval.optimizer.SingleImageDataset import SingleImageDataset
from search_eval.utils.common_utils import *
# from search_space.search_space import SearchSpace
from search_space.search_space_nodes import SearchSpace
from search_space.node_space import NodeSpace
from search_space.simple_space import SimpleSpace

from nni import trace
import nni.retiarii.strategy as strategy
import nni.retiarii.serializer as serializer

from nni.retiarii.experiment.pytorch import RetiariiExperiment, RetiariiExeConfig
from nni.retiarii.evaluator.pytorch import Lightning, Trainer
from nni.retiarii.evaluator.pytorch.lightning import DataLoader

import torch

torch.cuda.empty_cache()
dtype = torch.cuda.FloatTensor if torch.cuda.is_available() else torch.FloatTensor
print('CUDA available: {}'.format(torch.cuda.is_available()))

# Execute

In [ ]:

total_iterations = 35000

resolution = 64
noise_type = 'gaussian'
noise_level = '0.09'
phantom =       np.load(f'/home/joe/nas-for-dip/phantoms/ground_truth/{resolution}/{45}.npy')
phantom_noisy = np.load(f'/home/joe/nas-for-dip/phantoms/{noise_type}/res_{resolution}/nl_{noise_level}/p_{45}.npy')

# Create the lightning module
module = SGLDES(
                phantom=phantom, 
                phantom_noisy=phantom_noisy,
                
                learning_rate=0.001, # consider .01
                buffer_size=2000,
                patience=5000,
                weight_decay= 5e-7,

                show_every=500,
                report_every=100,
                HPO=False,
                NAS=True,
                OneShot=True,
                SGLD_regularize=True,
                switch=None
                )

# Create a PyTorch Lightning trainer
trainer = Trainer(
            max_epochs=total_iterations,
            fast_dev_run=False,
            gpus=1,
            )
            
if not hasattr(trainer, 'optimizer_frequencies'):
    trainer.optimizer_frequencies = []


# Create the lighting object for evaluator
train_loader = DataLoader(SingleImageDataset(phantom, num_iter=1), batch_size=1)
val_loader = DataLoader(SingleImageDataset(phantom, num_iter=1), batch_size=1)

lightning = Lightning(lightning_module=module, trainer=trainer, train_dataloaders=train_loader, val_dataloaders=val_loader)


# Select the Search Strategy
# search_strategy = strategy.DARTS()
# search_strategy = strategy.DartsStrategy()
# search_strategy = strategy.ENAS()
search_strategy = strategy.GumbelDARTS()
# search_strategy = strategy.RandomOneShot()

# # Create a Search Space
# model_space = SearchSpace(depth=4, enNodes=2)
depth = 1
nodes_per_layer = 1
ops_per_node = 1
poolOps_per_node = 1
upsampleOps_per_node = 1

model_space = SimpleSpace( #NodeSpace(
         depth=depth, 
         nodes_per_layer=nodes_per_layer,
         ops_per_node=ops_per_node, 
        #  poolOps_per_node=poolOps_per_node, 
        #  upsampleOps_per_node=upsampleOps_per_node
        )

# fast_dev_run=False

config = RetiariiExeConfig(execution_engine='oneshot')
experiment = RetiariiExperiment(model_space, evaluator=lightning, strategy=search_strategy)
experiment.run(config)

In [1]:
# stop experiment and clear cache
experiment.stop()
torch.cuda.empty_cache()

NameError: name 'experiment' is not defined

# testing

In [ ]:
# create a random tensor 1 1 64 64
# push the tensor through a nn.Conv2d(C_in, 64, kernel_size=3, padding=1) layer
# then push the output through a nn.MaxPool2d(kernel_size=2, stride=2, padding=0) layer
# print output shape

x = torch.rand(1, 1, 64, 64)
conv = torch.nn.Conv2d(1, 64, kernel_size=3, padding=1)
pool = torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
out = conv(x)
print(f'post in layer shape:   {out.shape}')
out = pool(out)
print(f'post pool layer shape: {out.shape}')

def conv_2d(C_in, C_out, kernel_size=3, dilation=1, padding=1, activation=None):
    return nn.Sequential(
        nn.Conv2d(C_in, C_out, kernel_size=kernel_size, dilation=dilation, padding=padding, bias=False),
        nn.BatchNorm2d(C_out),
        nn.ReLU() if activation is None else activation,
        nn.Conv2d(C_out, C_out, kernel_size=kernel_size, dilation=dilation, padding=padding, bias=False),
        nn.BatchNorm2d(C_out),
        nn.ReLU() if activation is None else activation
    )

conv = conv_2d(64, 128, kernel_size=3, padding=1)
out = conv(out)
print(f'post conv layer shape: {out.shape}')    

In [ ]:
# create a random tensor 1 1 64 64
# push the tensor through a nn.Conv2d(C_in, 64, kernel_size=3, padding=1) layer
# then push the output through a nn.MaxPool2d(kernel_size=2, stride=2, padding=0) layer
# print output shape

x = torch.rand(1, 1, 64, 64)
conv = torch.nn.Conv2d(1, 64, kernel_size=3, padding=1)
pool = torch.nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
out = conv(x)
out = pool(out)
print(out.shape)

def conv_2d(C_in, C_out, kernel_size=3, dilation=1, padding=1, activation=None):
    return nn.Sequential(
        nn.Conv2d(C_in, C_out, kernel_size=kernel_size, dilation=dilation, padding=padding, bias=False),
        nn.BatchNorm2d(C_out),
        nn.ReLU() if activation is None else activation,
        nn.Conv2d(C_out, C_out, kernel_size=kernel_size, dilation=dilation, padding=padding, bias=False),
        nn.BatchNorm2d(C_out),
        nn.ReLU() if activation is None else activation
    )

conv = conv_2d(64, 64, kernel_size=3, padding=1)
out = conv(out)
print(out.shape) 

In [ ]:

class ChannelAttention(nn.Module):
    def __init__(self, channel, reduction=16):
        super(ChannelAttention, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Sequential(
            nn.Linear(channel, channel // reduction, bias=False),
            nn.ReLU(inplace=True),
            nn.Linear(channel // reduction, channel, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        b, c, _, _ = x.size()
        y = self.avg_pool(x).view(b, c)
        y = self.fc(y).view(b, c, 1, 1)
        return x * y.expand_as(x)

def conv_2d_with_attention(C_in, C_out, kernel_size=3, dilation=1, padding=1, activation=None):
    return nn.Sequential(
        nn.Conv2d(C_in, C_out, kernel_size=kernel_size, dilation=dilation, padding=padding, bias=False),
        nn.BatchNorm2d(C_out),
        ChannelAttention(C_out),
        nn.ReLU() if activation is None else activation,
        nn.Conv2d(C_out, C_out, kernel_size=kernel_size, dilation=dilation, padding=padding, bias=False),
        nn.BatchNorm2d(C_out),
        ChannelAttention(C_out),
        nn.ReLU() if activation is None else activation
    )

x = torch.rand(1, 1, 64, 64)
conv = conv_2d_with_attention(1, 64, kernel_size=3, padding=1)
out = conv(x)
print(out.shape)